In [ ]:
### Generate p-values and networks for a large-scale benchmark study for 
### subnetwork discovery
## 1. Decide target subnetworks from database
## 2. load PPIs
## 3. Generate p-values and to q score, lfdr, gene lists...
## 4. send data to computing clusters


In [ ]:
## importing
import pandas as pd
import mygene
import numpy as np
import sys
import os
import math
from scipy.stats import beta, uniform, norm
from locfdr_python.locfdr_for_plot import locfdr
from statsmodels.stats.multitest import multipletests
from collections import OrderedDict
import subprocess
mg = mygene.MyGeneInfo()
## loading hugo names
hugo = pd.read_csv('/Users/leyang/Dropbox/Projects/Subnetwork_survey/code/Database/hugo_symbol.txt')
hugo = list(hugo['Approved symbol'])

In [ ]:
# Helper Functions
# writing networkx graph into file
def write_nx_to_file(G,index_file,edge_file,sif_file,tsv_file):
    gene_list = list(G.nodes())
    edge_list = list(G.edges())
    gene_list.sort()
    index_gene = dict(zip(gene_list,range(1,len(gene_list)+1)))
    edge_list_num = [(index_gene[edge[0]],index_gene[edge[1]]) for edge in edge_list]
    with open(index_file,'w') as f:
        for k in index_gene.keys():
            f.write('{}\t{}\n'.format(index_gene[k],k))
    with open(edge_file,'w') as f:
        for i in range(len(edge_list_num)):
            f.write('{}\t{}\t{}\n'.format(edge_list_num[i][0],edge_list_num[i][1],1))
    with open(sif_file,'w') as f:
        f.write('ID_interactor_A\tppi\tID_interactor_B\n')
        for i in range(len(edge_list)):
            f.write('{}\t{}\t{}\n'.format(edge_list[i][0],'ppi',edge_list[i][1]))
    with open(tsv_file,'w') as f:
        f.write('ID_interactor_A\tID_interactor_B\n')
        for i in range(len(edge_list)):
            f.write('{}\t{}\n'.format(edge_list[i][0],edge_list[i][1])) 

In [ ]:
## Part 1: Decide target subnetworks
## (a) load databases: CORUM and Reactome
corum_raw = pd.read_csv('/Users/leyang/Dropbox/Projects/Subnetwork_survey/code/Database/coreComplexes.txt',delimiter='\t')
reactome_raw = pd.read_csv('/Users/leyang/Dropbox/Projects/Subnetwork_survey/code/Database/human_pathway_lowlevel_ncbi.txt','\t')

## (b) pre-preocessing the complexes
corum_human = corum_raw[corum_raw['Organism'] == 'Human']
complex_ID = list(corum_human['ComplexID'])
complex_sub_entrez = list(corum_human['subunits(Entrez IDs)'])
complex_name = list(corum_human['ComplexName'])
complex_sub_gene_name = list(corum_human['subunits(Gene name)'])
complex_sub_entrez_split = [x.split(';') for x in complex_sub_entrez]
complex_sub_gene_name_split = [x.split(';') for x in complex_sub_gene_name]
complex_sub_size = [len(x) for x in complex_sub_entrez_split]
large_index = [k for k in range(len(complex_sub_size)) if complex_sub_size[k] >= 10]

complex_ID_large = [complex_ID[x] for x in large_index]
complex_name_large = [complex_name[x] for x in large_index]
complex_sub_entrez_split_large = [complex_sub_entrez_split[x] for x in large_index]
complex_sub_gene_name_split_large = [complex_sub_gene_name_split[x] for x in large_index]
complex_sub_size_large = [complex_sub_size[x] for x in large_index]

corum_tuple = zip(complex_ID_large, complex_name_large,complex_sub_size_large, complex_sub_entrez_split_large,complex_sub_gene_name_split_large)

## standarize gene names according to hugo symbol and entrez id
corum_standarize = []
for (i,name,s,entrez,gene) in corum_tuple:
    gene_list = []
    for (n,e) in zip(gene,entrez):
        g_tmp = 'None'
        if n not in hugo:
            out = mg.query(e, fields='symbol',species='human')
            if out['hits']:  
                symbol = out['hits'][0]['symbol']
                if symbol in hugo:
                    g_tmp = symbol
        else:
            g_tmp = n
        gene_list.append(g_tmp)
    corum_standarize.append((i,name,s,gene_list))

corum_standarize = [x for x in corum_standarize if 'None' not in x[3]]

remove_index = []
for i in range(len(corum_standarize)):
    for j in range(len(corum_standarize)):
        if i != j:
            intersect_length = len([val for val in corum_standarize[i][3] if val in corum_standarize[j][3]])
            if intersect_length / len(corum_standarize[i][3]) > 0.8:
                remove_index.append(i)

corum_final = [corum_standarize[i] for i in range(len(corum_standarize)) if i not in remove_index]
corum_final_save = [(i,name,s,','.join(gene_list)) for (i,name,s,gene_list) in corum_final]

## (c) pre-preocessing the reactome pathways
pathway_name = list(set(reactome_raw['pathway_name']))
reactome_standarize = []
for p in pathway_name:
    pp = reactome_raw[reactome_raw['pathway_name'] == p]
    gene_raw = list(pp['Gene'])
    gene_list = [gene_raw[i] for i in range(len(gene_raw)) if gene_raw[i] in hugo]
    evidence_raw = list(pp['evidence'])
    evidence_list = [evidence_raw[i] for i in range(len(gene_raw)) if gene_raw[i] in hugo]
    reactome_standarize.append((p,len(gene_list),gene_list,evidence_list))

reactome_standarize = [x for x in reactome_standarize if x[1] >= 10]

remove_index = []
for i in range(len(reactome_standarize)):
    for j in range(len(reactome_standarize)):
        if i != j:
            intersect_length = len([val for val in reactome_standarize[i][2] if val in reactome_standarize[j][2]])
            if intersect_length / len(reactome_standarize[i][3]) > 0.8:
                remove_index.append(i)
                
reactome_final = [reactome_standarize[i] for i in range(len(reactome_standarize)) if i not in set(remove_index)]
reactome_final_save = [(name,s,','.join(gene_list),','.join(evidence_list)) for (name,s,gene_list,evidence_list) in reactome_final]

df_corum = pd.DataFrame(corum_final_save,columns=['ID','Name','Size','Genes'])
df_corum.to_csv('./Database/corum.txt',sep='\t')
df_reactome = pd.DataFrame(reactome_final_save,columns=['Name','Size','Genes','Evidence'])
df_reactome.to_csv('./Database/reactome.txt',sep='\t')


In [ ]:
## Part 2: processing PPIs
# BIOGRID network: https://downloads.thebiogrid.org/File/BioGRID/Release-Archive/BIOGRID-4.4.212/BIOGRID-ALL-4.4.212.tab3.zip
# load raw file
ppi_biogrid = pd.read_table('./network_raw/BIOGRID-ALL-4.4.212.tab3.txt',header=0,sep='\t')

# extract homp sapien interactions: only the entry with both A and B are homo sapiens are extracted
tmp1 = ppi_biogrid['Organism Name Interactor A'] == 'Homo sapiens'
tmp2 = ppi_biogrid['Organism Name Interactor B'] == 'Homo sapiens'
tmp_index1 = [i for i in range(len(tmp1)) if tmp1[i]]
tmp_index2 = [i for i in range(len(tmp2)) if tmp2[i]]
homo_index = set(set(tmp_index1) & set(tmp_index2))

# entrez id of start and end
entrez_start = [ppi_biogrid['Entrez Gene Interactor A'][i] for i in homo_index]
entrez_end = [ppi_biogrid['Entrez Gene Interactor B'][i] for i in homo_index]

# build a ID converter using mygene: from entrez to offical symbols
IDs = set(entrez_start + entrez_end)
mg = mygene.MyGeneInfo()
# query all genes with the same IDtype from mygene, 14902 input query terms found dup hits (two hits are the same)
# 101 no hits, sample some and find them removed from NCBI
query_result = mg.querymany(IDs, scopes='entrezgene',fields='symbol',species='human',returnall=True) 
idconverter = dict([(x['query'],x['symbol']) for x in query_result['out'] if 'symbol' in x.keys()])

# manually annotated
idconverter['4549'] = 'MT-RNR1'
idconverter.pop('7795')

## build graph
biogrid = nx.Graph()
for (start,end) in zip(entrez_start,entrez_end):
    start = str(start)
    end = str(end)
    if start in idconverter.keys() and end in idconverter.keys():
        geneA = idconverter[start]
        geneB = idconverter[end]
        if geneA in approved_gene_list and geneB in approved_gene_list:
            biogrid.add_edge(geneA,geneB)
biogrid.remove_edges_from(list(nx.selfloop_edges(biogrid)))

print("BioGRID 4.4.212")
print("Number of Nodes:",len(biogrid.nodes()))
print("Number of Edges:",len(biogrid.edges()))
print("Degree of TP53:",biogrid.degree('TP53'))
print("Degree of PIK3CA:",biogrid.degree('PIK3CA'))
print("Degree of CDH1:",biogrid.degree('CDH1'))
## write index file, edge_list for FDRnet and multiplexFDRnet and sif and tsv for other methods
write_nx_to_file(biogrid,"biogrid_index_gene","./networks/biogrid_edge_list","./networks/biogrid_edge_list.sif","./networks/biogrid_edge_list.tsv")

# iRefIndex18 network: https://irefindex.vib.be/download/irefindex/data/archive/release_18.0/psi_mitab/MITAB2.6/
ppi_irefindex18 = pd.read_table('./network_raw/9606.mitab.06-11-2021.txt',header=0,sep='\t')
tmp = [x.startswith('complex:') for x in ppi_irefindex18['#uidA']]
index = [i for i in range(len(tmp)) if not tmp[i]] # remove complex entries

alias_start = [ppi_irefindex18['altA'][i].split('|')[0] for i in index] # get first alt
alias_end = [ppi_irefindex18['altB'][i].split('|')[0] for i in index]

# using entrez, refseq and uniprot id to get symbols
entrez_items = set([x.split(':')[1].strip() for x in alias_start if x.startswith('entrezgene/locuslink:')] + [x.split(':')[1].strip() for x in alias_end if x.startswith('entrezgene/locuslink:')]) #2158777
refseq_items = set([x.split(':')[1].strip() for x in alias_start if x.startswith('refseq:')] + [x.split(':')[1].strip() for x in alias_end if x.startswith('refseq:')]) # 173689
uniprot_items = set([x.split(':')[1].strip() for x in alias_start if x.startswith('uniprotkb:')] + [x.split(':')[1].strip() for x in alias_end if x.startswith('uniprotkb:')]) #90374


# build idconverters for entrez, uniprot and refseq
mg = mygene.MyGeneInfo()
query_entrez = mg.querymany(entrez_items, scopes='entrezgene',fields='symbol',species='human',returnall=True)
query_uniprot = mg.querymany(uniprot_items, scopes='uniprot',fields='symbol',species='human',returnall=True)
query_refseq = mg.querymany(refseq_items, scopes='refseq',fields='symbol',species='human',returnall=True)
idconverter_entrez = dict([(x['query'],x['symbol']) for x in query_entrez['out'] if 'symbol' in x.keys()])
idconverter_uniprot = dict([(x['query'],x['symbol']) for x in query_uniprot['out'] if 'symbol' in x.keys()])
idconverter_refseq = dict([(x['query'],x['symbol']) for x in query_refseq['out'] if 'symbol' in x.keys()])

# build graph
irefindex18 = nx.Graph()
for (start,end) in zip(alias_start,alias_end):
    #check which type the gene is annotated, get the id
    # geneA
    if start.startswith('entrez'):
        start = start[21:]
        if start in idconverter_entrez.keys():
            geneA = idconverter_entrez[start]
    elif start.startswith('uniprotkb:'):
        start = start[10:]
        if start in idconverter_uniprot.keys():
            geneA = idconverter_uniprot[start]
    elif start.startswith('refseq:'):
        start = start[7:]
        if start in idconverter_refseq.keys():
            geneA = idconverter_refseq[start]
    else:
        geneA = []
        
    # geneB
    if end.startswith('entrez'):
        end = end[21:]
        if end in idconverter_entrez.keys():
            geneB = idconverter_entrez[end]
    elif end.startswith('uniprotkb:'):
        end = end[10:]
        if end in idconverter_uniprot.keys():
            geneB = idconverter_uniprot[end]
    elif end.startswith('refseq:'):
        end = end[7:]
        if end in idconverter_refseq.keys():
            geneB = idconverter_refseq[end]
    else:
        geneB = []
    
    if geneA in approved_gene_list and geneB in approved_gene_list:
        irefindex18.add_edge(geneA,geneB)
irefindex18.remove_edges_from(list(nx.selfloop_edges(irefindex18)))

print("iRefindex 18.0")
print("Number of Nodes:",len(irefindex18.nodes()))
print("Number of Edges:",len(irefindex18.edges()))
print("Degree of TP53:",irefindex18.degree('TP53'))
print("Degree of PIK3CA:",irefindex18.degree('PIK3CA'))
print("Degree of CDH1:",irefindex18.degree('CDH1'))
write_nx_to_file(irefindex18,"irefindex18_index_gene","./networks/irefindex18_edge_list","./networks/irefindex18_edge_list.sif","./networks/irefindex18_edge_list.tsv")

# ReactomeFI 2021: https://reactome.org/download-data
ppi_reactome2021 = pd.read_table('./network_raw/FIsInGene_122921_with_annotations.txt',header=0,sep='\t')
start_gene = ppi_reactome2021['Gene1']
end_gene = ppi_reactome2021['Gene2']
reactome21 = nx.Graph()
for (start,end) in zip(start_gene,end_gene):
    if start in approved_gene_list and end in approved_gene_list:
        reactome21.add_edge(start,end)
        
reactome21.remove_edges_from(list(nx.selfloop_edges(reactome21)))
print("ReactomeFI 2021")
print("Number of Nodes:",len(reactome21.nodes()))
print("Number of Edges:",len(reactome21.edges()))
print("Degree of TP53:",reactome21.degree('TP53'))
print("Degree of PIK3CA:",reactome21.degree('PIK3CA'))
print("Degree of CDH1:",reactome21.degree('CDH1'))
write_nx_to_file(reactome21,"reactome21_index_gene","./networks/reactome21_edge_list","./networks/reactome21_edge_list.sif","./networks/reactome21_edge_list.tsv")

# STRING v11.0:
ppi_string = pd.read_table('./network_raw/9606.protein.links.v11.5.txt',header=0,sep=' ')
ppi_string_annotation = pd.read_table('./network_raw/9606.protein.info.v11.5.txt',header=0,sep='\t')
high_score = ppi_string['combined_score'] >= 900
high_index = [i for i in range(len(high_score)) if high_score[i]]

# id converted from string database
ppi_string_annotation = pd.read_table('./network_raw/9606.protein.info.v11.5.txt',header=0,sep='\t')
idconverter_string = dict(zip(ppi_string_annotation['#string_protein_id'],ppi_string_annotation['preferred_name'])) 


# get high confidence ensemble ids
start_ensembl = [ppi_string['protein1'][i] for i in high_index]
end_ensembl = [ppi_string['protein2'][i] for i in high_index]

string_900 = nx.Graph()
for (start,end) in zip(start_ensembl,end_ensembl):
    if start in idconverter_string.keys() and end in idconverter_string.keys():
        geneA = idconverter_string[start]
        geneB = idconverter_string[end]
        if geneA in approved_gene_list and geneB in approved_gene_list:
            string_900.add_edge(geneA,geneB)

string_900.remove_edges_from(list(nx.selfloop_edges(string_900)))
print("STRING v11 high confidence (900)")
print("Number of Nodes:",len(string_900.nodes()))
print("Number of Edges:",len(string_900.edges()))
print("Degree of TP53:",string_900.degree('TP53'))
print("Degree of PIK3CA:",string_900.degree('PIK3CA'))
print("Degree of CDH1:",string_900.degree('CDH1'))
write_nx_to_file(string_900,"string_index_gene","./networks/string_edge_list","./networks/string_edge_list.sif","./networks/string_edge_list.tsv")

# merge biogrid, irefindex18, reactome21, string_900
merge_bioirefreastr = nx.compose(nx.compose(nx.compose(biogrid,irefindex18),reactome21),string_900)

print("merge_bioirefreastr")
print("Number of Nodes:",len(merge_bioirefreastr.nodes()))
print("Number of Edges:",len(merge_bioirefreastr.edges()))
print("Degree of TP53:",merge_bioirefreastr.degree('TP53'))
print("Degree of PIK3CA:",merge_bioirefreastr.degree('PIK3CA'))
print("Degree of CDH1:",merge_bioirefreastr.degree('CDH1'))
write_nx_to_file(merge_bioirefreastr,"./networks/merge_bioirefreastr_index_gene","./networks/merge_bioirefreastr_edge_list","./networks/merge_bioirefreastr_edge_list.sif","./networks/merge_bioirefreastr_edge_list.tsv")


In [ ]:
## Part 3: generate p-values
## (a) p-values for CORUM
## (a).1 sampling target subnetworks
## (a).2 sampling p-values 110 times, a = 0.01 to 0.11, 
## (a).3 transform p-values to q-score, locfdr, gene list, etc
## (a).4 save all data
## (a).5 send all data to CCR
np.random.seed(seed=42)
#load genes
tmp = pd.read_csv('./networks/merge_bioirefreastr_index_gene',sep='\t',header=None)
total_gene = list(tmp[1])
tmp_bio = pd.read_csv('./networks/biogrid_index_gene',sep='\t',header=None)
bio_gene = list(tmp_bio[1])
tmp_iref = pd.read_csv('./networks/irefindex18_index_gene',sep='\t',header=None)
iref_gene = list(tmp_iref[1])
tmp_rea = pd.read_csv('./networks/reactome21_index_gene',sep='\t',header=None)
rea_gene = list(tmp_rea[1])
tmp_str = pd.read_csv('./networks/string_index_gene',sep='\t',header=None)
str_gene = list(tmp_str[1])

# CORUM
base_path = '/Users/leyang/Dropbox/Projects/Subnetwork_survey'
for i in range(5):
    name = "corum_" + str(i)
    path = os.path.join(base_path,'code','score',name)
    path_p = os.path.join(base_path,'code','score',name,'p')
    path_q = os.path.join(base_path,'code','score',name,'q')
    path_z = os.path.join(base_path,'code','score',name,'z')
    path_lfdr = os.path.join(base_path,'code','score',name,'lfdr')
    path_list = os.path.join(base_path,'code','score',name,'list')
    path_clustex = os.path.join(base_path,'code','score',name,'clustex')
    path_diamond = os.path.join(base_path,'code','score',name,'diamond')
    path_all = [path_p,path_q,path_z,path_lfdr,path_list,path_clustex,path_diamond]
    if not os.path.exists(path):
        os.makedirs(path)
    for pa in path_all:
        if not os.path.exists(pa):
            os.makedirs(pa)
    index = np.random.choice(len(corum_final),size=10,replace=False)
    corum_target = [corum_final[x][3] for x in range(len(corum_final)) if x in index]
    corum_target_save = [corum_final_save[x] for x in range(len(corum_final_save)) if x in index]
    tmp_df = pd.DataFrame(corum_target_save,columns=['ID','Name','Size','Genes'])
    tmp_df.to_csv(os.path.join(path,name+'_targets.txt'),sep='\t',index=False) #save targets
    target_genes = set([x for l in corum_target for x in l])
    
    target = [x for x in target_genes if x in total_gene]
    non_target = [x for x in total_gene if x not in target]
    a_list = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11]
    for a in a_list:
        for permu in range(1,11):
            
            target_p = beta.rvs(a=a, b=1, size=len(target))
            p_dict = dict(zip(target,target_p))
            non_target_p = uniform.rvs(size=len(non_target))
            non_target_dict = dict(zip(non_target,non_target_p))
            p_dict.update(non_target_dict)
            
            # BH method to p_adjusted, then -log10 to qvalues
            p_vals = [p_dict[k] for k in p_dict.keys()]
            tmp,p_adjusted,tmp,tmp = multipletests(p_vals,method='fdr_bh')
            q_score = [-math.log10(p) if p!=0 else 200 for p in p_adjusted]
            q_dict = dict(zip(p_dict.keys(),q_score))
            
            # calculate z-values
            z_vals = [-norm.ppf(p) for p in p_vals]
            z_dict = dict(zip(p_dict.keys(),z_vals))
            
            # calculate lfdr
            
            index_noninfs = [i for i in range(len(z_vals)) if z_vals[i] != -float('Inf') and z_vals[i] != float('Inf')]
            results = locfdr([z_vals[i] for i in index_noninfs],bre=120,df=10,saveplot=True,saveroot='./locfdr_plot/'+name+'beta_'+str(a)+'_'+str(permu),showplot=False)
            fdr_noninfs = results['fdr']
            fdr_dict = dict(zip(index_noninfs,fdr_noninfs))
            fdr = [1.0 for i in range(len(z_vals))]
            for i in range(len(z_vals)):
                if i in index_noninfs:
                    fdr[i] = fdr_dict[i]
                elif z_vals[i] == float('Inf'):
                    fdr[i] = 0
            for i in range(len(fdr)):
                if z_vals[i] < 0:
                    fdr[i] = 1.0
                if (z_vals[i] > 0) and math.isnan(fdr[i]):
                    fdr[i] = 0.0
            lfdr_dict = dict(zip(p_dict.keys(),fdr))
            
            # gene list
            gene_list = [k for k,v in lfdr_dict.items() if v <= 0.1]
            gene_list_index_bio = [bio_gene.index(g) + 1 for g in gene_list if g in bio_gene]
            gene_list_index_iref = [iref_gene.index(g) + 1 for g in gene_list if g in iref_gene]
            gene_list_index_rea = [rea_gene.index(g) + 1 for g in gene_list if g in rea_gene]
            gene_list_index_str = [str_gene.index(g) + 1 for g in gene_list if g in str_gene]
            
            
            # save all scores to files
            p_df = pd.DataFrame(sorted(p_dict.items()),columns=['Gene','p'])
            p_df.to_csv(os.path.join(path_p,name+'_p_beta_'+str(a)+'_'+str(permu)+'.txt'),header=None,sep='\t',index=False)
            q_df = pd.DataFrame(sorted(q_dict.items()),columns=['Gene','q'])
            q_df.to_csv(os.path.join(path_q,name+'_q_beta_'+str(a)+'_'+str(permu)+'.txt'),header=None,sep='\t',index=False)
            z_df = pd.DataFrame(sorted(z_dict.items()),columns=['Gene','z'])
            z_df.to_csv(os.path.join(path_z,name+'_z_beta_'+str(a)+'_'+str(permu)+'.txt'),header=None,sep='\t',index=False)
            lfdr_df = pd.DataFrame(sorted(lfdr_dict.items()),columns=['Gene','lfdr'])
            lfdr_df.to_csv(os.path.join(path_lfdr,name+'_lfdr_beta_'+str(a)+'_'+str(permu)+'.txt'),header=None,sep='\t',index=False)
            
            # gene list
            with open(os.path.join(path_list,name+'_list_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                for g in gene_list:
                    file.write("%s\n" % g)
                    
            # clustex biogrid
            with open(os.path.join(path_clustex,name+'_clustex_biogrid_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                file.write("Gene\n")
                for g in gene_list_index_bio:
                    file.write("%s\n" % g)
            # clustex irefindex
            with open(os.path.join(path_clustex,name+'_clustex_irefindex18_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                file.write("Gene\n")
                for g in gene_list_index_iref:
                    file.write("%s\n" % g)
            # clustex reactome21
            with open(os.path.join(path_clustex,name+'_clustex_reactome21_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                file.write("Gene\n")
                for g in gene_list_index_rea:
                    file.write("%s\n" % g)
            # clustex string
            with open(os.path.join(path_clustex,name+'_clustex_string_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                file.write("Gene\n")
                for g in gene_list_index_str:
                    file.write("%s\n" % g)
                    
            # diamond biogrid
            with open(os.path.join(path_diamond,name+'_diamond_biogrid_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                for g in gene_list_index_bio:
                    file.write("%s\n" % g)
            # diamond irefindex
            with open(os.path.join(path_diamond,name+'_diamond_irefindex18_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                for g in gene_list_index_iref:
                    file.write("%s\n" % g)
            # diamond reactome21
            with open(os.path.join(path_diamond,name+'_diamond_reactome21_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                for g in gene_list_index_rea:
                    file.write("%s\n" % g)
            # diamond string
            with open(os.path.join(path_diamond,name+'_diamond_string_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                for g in gene_list_index_str:
                    file.write("%s\n" % g)
                    
## Reactome
for i in range(5):
    name = "reactome_" + str(i)
    path = os.path.join(base_path,'code','score',name)
    path_p = os.path.join(base_path,'code','score',name,'p')
    path_q = os.path.join(base_path,'code','score',name,'q')
    path_z = os.path.join(base_path,'code','score',name,'z')
    path_lfdr = os.path.join(base_path,'code','score',name,'lfdr')
    path_list = os.path.join(base_path,'code','score',name,'list')
    path_clustex = os.path.join(base_path,'code','score',name,'clustex')
    path_diamond = os.path.join(base_path,'code','score',name,'diamond')
    path_all = [path_p,path_q,path_z,path_lfdr,path_list,path_clustex,path_diamond]
    if not os.path.exists(path):
        os.makedirs(path)
    for pa in path_all:
        if not os.path.exists(pa):
            os.makedirs(pa)
    index = np.random.choice(len(reactome_final),size=10,replace=False)
    reactome_target = [reactome_final[x][2] for x in range(len(reactome_final)) if x in index]
    reactome_target_save = [reactome_final_save[x] for x in range(len(reactome_final_save)) if x in index]
    tmp_df = pd.DataFrame(reactome_target_save,columns=['ID','Name','Size','Genes'])
    tmp_df.to_csv(os.path.join(path,name+'_targets.txt'),sep='\t',index=False) #save targets
    target_genes = set([x for l in reactome_target for x in l])
    
    target = [x for x in target_genes if x in total_gene]
    non_target = [x for x in total_gene if x not in target]
    a_list = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11]
    for a in a_list:
        for permu in range(1,11):
            
            target_p = beta.rvs(a=a, b=1, size=len(target))
            p_dict = dict(zip(target,target_p))
            non_target_p = uniform.rvs(size=len(non_target))
            non_target_dict = dict(zip(non_target,non_target_p))
            p_dict.update(non_target_dict)
            
            # BH method to p_adjusted, then -log10 to qvalues
            p_vals = [p_dict[k] for k in p_dict.keys()]
            tmp,p_adjusted,tmp,tmp = multipletests(p_vals,method='fdr_bh')
            q_score = [-math.log10(p) if p!=0 else 200 for p in p_adjusted]
            q_dict = dict(zip(p_dict.keys(),q_score))
            
            # calculate z-values
            z_vals = [-norm.ppf(p) for p in p_vals]
            z_dict = dict(zip(p_dict.keys(),z_vals))
            
            # calculate lfdr
            
            index_noninfs = [i for i in range(len(z_vals)) if z_vals[i] != -float('Inf') and z_vals[i] != float('Inf')]
            results = locfdr([z_vals[i] for i in index_noninfs],bre=120,df=10,saveplot=True,saveroot='./locfdr_plot/'+name+'beta_'+str(a)+'_'+str(permu),showplot=False)
            fdr_noninfs = results['fdr']
            fdr_dict = dict(zip(index_noninfs,fdr_noninfs))
            fdr = [1.0 for i in range(len(z_vals))]
            for i in range(len(z_vals)):
                if i in index_noninfs:
                    fdr[i] = fdr_dict[i]
                elif z_vals[i] == float('Inf'):
                    fdr[i] = 0
            for i in range(len(fdr)):
                if z_vals[i] < 0:
                    fdr[i] = 1.0
                if (z_vals[i] > 0) and math.isnan(fdr[i]):
                    fdr[i] = 0.0
            lfdr_dict = dict(zip(p_dict.keys(),fdr))
            
            # gene list
            gene_list = [k for k,v in lfdr_dict.items() if v <= 0.1]
            gene_list_index_bio = [bio_gene.index(g) + 1 for g in gene_list if g in bio_gene]
            gene_list_index_iref = [iref_gene.index(g) + 1 for g in gene_list if g in iref_gene]
            gene_list_index_rea = [rea_gene.index(g) + 1 for g in gene_list if g in rea_gene]
            gene_list_index_str = [str_gene.index(g) + 1 for g in gene_list if g in str_gene]
            
            
            # save all scores to files
            p_df = pd.DataFrame(sorted(p_dict.items()),columns=['Gene','p'])
            p_df.to_csv(os.path.join(path_p,name+'_p_beta_'+str(a)+'_'+str(permu)+'.txt'),header=None,sep='\t',index=False)
            q_df = pd.DataFrame(sorted(q_dict.items()),columns=['Gene','q'])
            q_df.to_csv(os.path.join(path_q,name+'_q_beta_'+str(a)+'_'+str(permu)+'.txt'),header=None,sep='\t',index=False)
            z_df = pd.DataFrame(sorted(z_dict.items()),columns=['Gene','z'])
            z_df.to_csv(os.path.join(path_z,name+'_z_beta_'+str(a)+'_'+str(permu)+'.txt'),header=None,sep='\t',index=False)
            lfdr_df = pd.DataFrame(sorted(lfdr_dict.items()),columns=['Gene','lfdr'])
            lfdr_df.to_csv(os.path.join(path_lfdr,name+'_lfdr_beta_'+str(a)+'_'+str(permu)+'.txt'),header=None,sep='\t',index=False)
            
            # gene list
            with open(os.path.join(path_list,name+'_list_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                for g in gene_list:
                    file.write("%s\n" % g)
                    
            # clustex biogrid
            with open(os.path.join(path_clustex,name+'_clustex_biogrid_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                file.write("Gene\n")
                for g in gene_list_index_bio:
                    file.write("%s\n" % g)
            # clustex irefindex
            with open(os.path.join(path_clustex,name+'_clustex_irefindex18_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                file.write("Gene\n")
                for g in gene_list_index_iref:
                    file.write("%s\n" % g)
            # clustex reactome21
            with open(os.path.join(path_clustex,name+'_clustex_reactome21_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                file.write("Gene\n")
                for g in gene_list_index_rea:
                    file.write("%s\n" % g)
            # clustex string
            with open(os.path.join(path_clustex,name+'_clustex_string_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                file.write("Gene\n")
                for g in gene_list_index_str:
                    file.write("%s\n" % g)
                    
            # diamond biogrid
            with open(os.path.join(path_diamond,name+'_diamond_biogrid_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                for g in gene_list_index_bio:
                    file.write("%s\n" % g)
            # diamond irefindex
            with open(os.path.join(path_diamond,name+'_diamond_irefindex18_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                for g in gene_list_index_iref:
                    file.write("%s\n" % g)
            # diamond reactome21
            with open(os.path.join(path_diamond,name+'_diamond_reactome21_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                for g in gene_list_index_rea:
                    file.write("%s\n" % g)
            # diamond string
            with open(os.path.join(path_diamond,name+'_diamond_string_beta_'+str(a)+'_'+str(permu)+'.txt'),'w') as file:
                for g in gene_list_index_str:
                    file.write("%s\n" % g)
         
                    
                    
    


In [ ]:
## send networks and scores to CCR
data_version_list = ["corum_"+str(i) for i in range(5)] +  ["reactome_"+str(i) for i in range(5)]
beta = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11]
permu = [1,2,3,4,5,6,7,8,9,10]
network_list = ["biogrid","irefindex18","reactome21","string"]

## data
fdrdata = [os.path.join("./","score",data_version,"lfdr",data_version+"_lfdr_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list]
pdata = [os.path.join("./","score",data_version,"p",data_version+"_p_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list]
qdata = [os.path.join("./","score",data_version,"q",data_version+"_q_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list]
zdata = [os.path.join("./","score",data_version,"z",data_version+"_z_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list]
genelistdata = [os.path.join("./","score",data_version,"list",data_version+"_list_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list]
clustexdata = [os.path.join("./","score",data_version,"clustex",data_version+"_clustex_"+network+"_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list for network in network_list]
diamonddata = [os.path.join("./","score",data_version,"diamond",data_version+"_diamond_"+network+"_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list for network in network_list]
## network
network_index_gene = [os.path.join("./","networks",network+"_index_gene") for network in network_list]
network_edge_list = [os.path.join("./","networks",network+"_edge_list") for network in network_list]
network_edge_sif = [os.path.join("./","networks",network+"_edge_list.sif") for network in network_list]
network_edge_tsv = [os.path.join("./","networks",network+"_edge_list.tsv") for network in network_list]
network_clustex = [os.path.join("./","networks",network+"_edge_list_clustex.txt") for network in network_list]
## ensure data exists
for f in fdrdata + pdata + qdata + zdata + genelistdata + clustexdata + diamonddata + network_index_gene + network_edge_list + network_edge_sif + network_edge_tsv:
    assert os.path.exists(f)

In [ ]:
#send to domino
subprocess.run(["scp",*network_edge_sif,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/domino/network/"])
subprocess.run(["scp",*genelistdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/domino/data/"])


In [ ]:
# send to diamond
subprocess.run(["scp",*network_edge_list,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/diamond/network/"])
subprocess.run(["scp",*diamonddata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/diamond/data/"])


In [ ]:
# send to must
# network
subprocess.run(["scp",*network_edge_tsv,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/must/network/"])
subprocess.run(["scp",*genelistdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/must/data/"])


In [ ]:
# send to clustex
# network
subprocess.run(["scp",*network_clustex,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/clustex/network/"])
subprocess.run(["scp",*clustexdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/clustex/data/"])


In [ ]:
# send to netcore
# network
subprocess.run(["scp",*network_edge_tsv,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/netcore/network/"])
subprocess.run(["scp",*genelistdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/netcore/data/"])


In [ ]:
# send to robust
# network
subprocess.run(["scp",*network_edge_tsv,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/robust/network/"])
subprocess.run(["scp",*genelistdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/robust/data/"])



In [ ]:
# send to BioNet
# network
#subprocess.run(["scp",*network_index_gene,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/bionet/network/"])
#subprocess.run(["scp",*network_edge_list,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/bionet/network/"])
# score
subprocess.run(["scp",*pdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/bionet/data/"])
# send to hotnet2
subprocess.run(["scp",*qdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/hotnet2/survey/data/heats/"])
# send to netmix2
subprocess.run(["scp",*pdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/netmix2/data/"])
#subprocess.run(["scp",*network_edge_tsv,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/netmix2/network/"])
# send to FDRnet
subprocess.run(["scp",*fdrdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/fdrnet/data/"])
# network




In [ ]:

subprocess.run(["scp",*network_index_gene,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/fdrnet/network/"])
subprocess.run(["scp",*network_edge_list,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/fdrnet/network/"])

In [ ]:
# send to FDRnet
subprocess.run(["scp",*fdrdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/data/"])

# send to hhotnet
subprocess.run(["scp",*qdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/hhotnet/biogrid/Data/"])

# send to netmix2
subprocess.run(["scp",*pdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/netmix2/data/"])

# send to domino
subprocess.run(["scp",*genelistdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/domino/Data/"])

# send to BioNet
subprocess.run(["scp",*pdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/bionet/data/"])

# send to clustex
subprocess.run(["scp",*clustexdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/clustex/data/"])

# send to diamond
subprocess.run(["scp",*diamonddata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/diamond/data/"])


In [ ]:
# # send to regmod
# subprocess.run(["scp",*data_mat,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey_others/regmod/data/"])


In [ ]:
data_path = "./data";
network_path = "./network"
result_folder = "./result"
network_list = ["biogrid","irefindex18","reactome21","string"]
network = network_list*1100#[net for net in network_list for i in range(1100)]
data_version_list = ["corum_"+str(k) for k in range(5)] + ["reactome_"+str(k) for k in range(5)]
beta = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11]
permu = [1,2,3,4,5,6,7,8,9,10]
score = ["".join((data_version,"_diamond_"+net+"_beta_",str(b),"_",str(p))) for b in beta for p in permu for data_version in data_version_list for net in network_list]
#python run_netmix2.py -el data/edge_list.tsv -gs data/gene_scores.tsv -o results/example_output.tsv

for n in range(4400):
    assert network[n] in score[n]
